# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd

import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")


In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
print(data.info())
print(data.head())
print(data.describe())
print(data.corr())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None
   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0
               Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
count  5000.000000  5000.000000   5000.000000  5000.000000        5000.0000

In [4]:
data.columns = ['sex', 'age', 'income', 'family', 'unsurance_pay']
data

,sex,age,income,family,unsurance_pay
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [5]:
features = data.drop(['unsurance_pay'], axis=1)
features_columns=features.columns
target = data['unsurance_pay']

In [6]:
train_features, test_features, train_target, test_target = (
train_test_split(features, target, test_size =0.2, random_state=1245)) 

Получены данные о клиетах страховой компани  содержащие персональные данные (возраст и доход) а так же обезличенные данные: количественный состав семьи, пол, количество страховых выплат за год. Данные не имеют пропусков. Изменены названия колонок, данные разделены на целевые признаки (пол, возраст, доход, количественный состав семьи) и конечный результат (количество страховых выплат в год). Есть средняя зависимость между возрастом клиента и количеством страховых выплат.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $w1$ - случайный вектор линейной регрессии (нулевой элемент равен сдвигу)

- $a*$ - предсказания модели с умноженной на обратимую матрицу матрицей целевого признака

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** умножение матрицы признаков на обратимую матрицу не изменяет предсказания

**Обоснование:**
Умножим матрицу признаков (X) на обратимую матрицу (P) высота которой равна ширине матрицы (X) 

Предсказания:

$$
a* = XPw1
$$

Формула обучения при умножении целевого признака на матрицу P:

$$
w1 = (XP^T)  XP)^{-1} (XP)^T y
$$

Раскроем транспонирование матриц
$$
w1 = (X^T P^T  XP)^{-1} X^T P^T y
$$

Раскроем скобки для обратных матриц и перегруппируем члены уравнения

$$
w1 = P^{-1} (X^TX)^{-1} (P^T)^{-1} P^T X^T  y
$$

По условию матриц P оборатимая и по свойству оборатимых матриц выполняется равенство $P*P^{-1}=E$
$$
w1 = P^{-1} (X^T X)^{-1} E  X^T  y
$$


Упростим полученное уравнение:<br>  
базовое уравнение обучения
$$w = (X^T X)^{-1} X^T y$$ 

таким образом полученное ранее уравнение можно представить в виде:
$$
w1 =P^{-1} w
$$

Заменим $w1$ в формуле предсказаний на  $P^{-1} w$:

$$
a* = XPP^{-1} w
$$

По условию матриц P оборатимая и по свойству оборатимых матриц выполняется равенство $P*P^{-1}=E$
$$
a* = XEw = Xw = a
$$
то есть предсказания базовой модели и модели в которой целевые признаки умножены на любую обратимую матрицу остаются без изменений
$$
a*=a
$$

## Алгоритм преобразования

**Алгоритм**<br>
<b>1. Умножение матрицы признаков на случайную обратимую матрицу</b><br>
Генерирование случайной обратимой матрицы с размерами равным ширине массива целевых признаков<br>
$$a = XPw$$
<b>2. Стандартизация данных</b><br>
Преобразование данных в столбцах Возраст и Зарплата на величину стандартного отклонения по соответсвующей совокупности данных(аналог StandartScaler из библиотеки sklearn)<br> $$ z = (x-m)/q$$ где x - истинное значение, m-среднее выборки, q - стандартное отклонение выборки

<b>1. Умножение матрицы признаков на случайную обратимую матрицу</b><br>

In [7]:
random_matrix = np.random.normal(size=(4,4))
random_matrix

array([[-0.11104853, -0.22513506, -1.20098292, -0.24055427],
       [-0.99512464, -1.63018285, -0.86655146, -1.26063985],
       [ 0.40236909,  1.36627381,  0.43045329, -0.30180381],
       [-1.24636922, -2.02479073,  1.14092516, -0.2193782 ]])

In [8]:
encoded_features_matrix = features.dot(random_matrix)
encoded_features_matrix

,0,1,2,3
0,19915.349395,67698.093533,21314.894360,-15021.615229
1,15243.003363,51841.391563,16318.504458,-11526.753658
2,8420.892301,28644.474699,9014.389032,-6374.438602
3,16755.400747,56935.334439,17933.986331,-12612.131143
4,10473.858742,35613.876176,11209.366363,-7912.617957
...,...,...,...,...
4995,14334.220327,48726.280301,15345.200750,-10810.132752
4996,21049.059772,71535.296615,22527.430407,-15857.600869
4997,13617.916960,46280.028907,14577.317245,-10256.800772
4998,13131.726384,44634.990044,14058.980141,-9897.617410


In [9]:
deencoded_features_matrix=round(encoded_features_matrix.dot(np.linalg.inv(random_matrix)))
deencoded_features_matrix

,0,1,2,3
0,1.0,41.0,49600.0,1.0
1,-0.0,46.0,38000.0,1.0
2,0.0,29.0,21000.0,0.0
3,-0.0,21.0,41700.0,2.0
4,1.0,28.0,26100.0,-0.0
...,...,...,...,...
4995,-0.0,28.0,35700.0,2.0
4996,-0.0,34.0,52400.0,1.0
4997,0.0,20.0,33900.0,2.0
4998,1.0,22.0,32700.0,3.0


<b>2. Стандартизация данных</b><br>

In [10]:
age_vector = features['age'].values
income_vector = features['income'].values
encoded_age_vector = pd.Series((age_vector-np.mean(age_vector))/np.std(age_vector))
encoded_income_vector = pd.Series((income_vector-np.mean(income_vector))/np.std(income_vector))
encoded_age_vector.index=features.index
encoded_income_vector.index=features.index

In [11]:
features['age_encode']=encoded_age_vector
features['income_encode']=encoded_income_vector
features_encode_standart = features.drop(['age', 'income'], axis=1)

In [12]:
features.drop(['age_encode', 'income_encode'], axis=1, inplace=True)

In [13]:
print(features)

      sex   age   income  family
0       1  41.0  49600.0       1
1       0  46.0  38000.0       1
2       0  29.0  21000.0       0
3       0  21.0  41700.0       2
4       1  28.0  26100.0       0
...   ...   ...      ...     ...
4995    0  28.0  35700.0       2
4996    0  34.0  52400.0       1
4997    0  20.0  33900.0       2
4998    1  22.0  32700.0       3
4999    1  28.0  40600.0       1

[5000 rows x 4 columns]


**Обоснование**


В разделе 2 представлен вывод о неизменности предсказаний при умножении матрицы целевых признаков на обратимую матрицу. Представленная операция кодирования целевых признаков посредством стандартизации данных является частным вариантом умножения целевых признаков на обратимую матрицу. Проверим это опытным путем.

In [14]:
model=LinearRegression()
model.fit(train_features, train_target)
original_data_pred = model.predict(test_features)
r2_score_original_data = r2_score(test_target, original_data_pred)
print(r2_score_original_data)

0.42190741870614445


In [15]:
encoded_train_features_matrix, encoded_test_features_matrix, train_target, test_target = (
train_test_split(encoded_features_matrix, target, test_size =0.2, random_state=1245)) 

In [16]:
model.fit(encoded_train_features_matrix, train_target)
matrix_data_pred = model.predict(encoded_test_features_matrix)
r2_score_matrix_data = r2_score(test_target, matrix_data_pred)
print(r2_score_matrix_data)


0.4219074187058621


In [17]:
train_features_encode_standart, test_features_encode_standart, train_target, test_target = (
train_test_split(features_encode_standart, target, test_size =0.2, random_state=1245)) 

In [18]:
model.fit(train_features_encode_standart, train_target)
standart_data_pred = model.predict(test_features_encode_standart)
r2_score_standart_data = r2_score(test_target, standart_data_pred)
print(r2_score_standart_data)


0.421907418706137


Метрики R2 совпадают для предложенных вариантов

## Проверка алгоритма

In [19]:
train_features, test_features, train_target, test_target = (
train_test_split(features, target, test_size =0.2, random_state=12345)) 

In [20]:
model.fit(train_features, train_target)
original_data_pred = model.predict(test_features)
r2_score_original_data = r2_score(test_target, original_data_pred)
print(r2_score_original_data)

0.41176839567705226


Запишем в виде функции алгоритм умножения матрицы целевых признаков 

In [21]:
def encode_features_matrix (features):
    size_matrix = features.shape[1]
    random_matrix = np.random.normal(size=(size_matrix,size_matrix))
    return features.dot(random_matrix)

In [22]:
encode_features_matrix1=encode_features_matrix(features)

In [23]:
encoded_train_features_matrix1, encoded_test_features_matrix1, train_target, test_target = (
train_test_split(encode_features_matrix1, target, test_size =0.2, random_state=12345)) 

In [24]:
model.fit(encoded_train_features_matrix1, train_target)
matrix_data_pred1 = model.predict(encoded_test_features_matrix1)
r2_score_matrix_data1 = r2_score(test_target, matrix_data_pred1)
print(r2_score_matrix_data1)

0.411768395676956


Запишем в виде функции алгоритм кодирования защищаемых данных (дохода и возраста) посредством их стандартизации (сравнения отклонения от среднего со стандартным отклонением выборки)

In [25]:
def encode_features_standart(features):
    age_vector = features['age'].values
    income_vector = features['income'].values
    encoded_age_vector = pd.Series((age_vector-np.mean(age_vector))/np.std(age_vector))
    encoded_income_vector = pd.Series((income_vector-np.mean(income_vector))/np.std(income_vector))
    encoded_age_vector.index=features.index
    encoded_income_vector.index=features.index
    features['age_encode']=encoded_age_vector
    features['income_encode']=encoded_income_vector
    features_encode_standart = features.drop(['age', 'income'], axis=1)
    features.drop(['age_encode', 'income_encode'], axis=1, inplace=True)
    return features_encode_standart

In [26]:
encode_features_standart1=encode_features_standart(features)

In [27]:
train_features_encode_standart1, test_features_encode_standart1, train_target, test_target = (
train_test_split(encode_features_standart1, target, test_size =0.2, random_state=12345)) 

In [28]:
model.fit(train_features_encode_standart1, train_target)
standart_data_pred1 = model.predict(test_features_encode_standart1)
r2_score_standart_data1 = r2_score(test_target, standart_data_pred1)
print(r2_score_standart_data1)

0.41176839567704737


In [29]:
print('Метрика R2 для незакодированных данных: ', r2_score_original_data)
print('Метрика R2 при умножении целевых признаков на случайную обратимую матрицу: '
      ,r2_score_matrix_data1)
print('Метрика R2 при стандартизации защищаемых признаков (дохода и возраста): '
      ,r2_score_standart_data1)

Метрика R2 для незакодированных данных:  0.41176839567705226
Метрика R2 при умножении целевых признаков на случайную обратимую матрицу:  0.411768395676956
Метрика R2 при стандартизации защищаемых признаков (дохода и возраста):  0.41176839567704737


Вывод:
Предложенные способы кодирования целевых признаков или части целевых признаков работают. Метрики R2 для данных закодированных предложенными способами существенно не изменяются.

Выводы по проекту:
1. Получены данные о клиентах страховой компании содержащие пол, доход, возраст, количественный состав семьи, и количество страховых выплат.
2. Теоретически обосновано умножение матрицы признаков на случайную обратимую матрицу.
3. Предложены модели кодирования данных которые дают равные показания метрики R2 не незакодированных и закодированных данных.
4. Предложенные алгоритмы отражены в функциях и опробированы на имеющихся данных. Метрики R2 существенно не отличаются.